# Experiment 03 - Data

Create data splits

In [ ]:
from dataclasses import dataclass
import logging
import typing
from pathlib import Path
from pprint import pprint
from src.nb_imports import *
pd.options.display.max_columns = 32

import plotly.express as px

from omegaconf import OmegaConf
from sklearn.neighbors import NearestNeighbors
from vaep.pandas import interpolate, parse_query_expression
from vaep.io.datasplits import DataSplits
from vaep.sampling import feature_frequency, frequency_by_index, sample_data

import src
from vaep.logging import setup_logger
logger = setup_logger(logger=logging.getLogger('vaep'))
logger.info("Experiment 03 - data")

figures = {}  # collection of ax or figures

## Arguments

In [ ]:
FN_PEPTIDE_INTENSITIES: str = 'data/df_intensities_N07813_M10000.csv'  # Samples metadata extraced from erda
FN_PEPTIDE_FREQ: str = 'data/processed/count_all_peptides.json' # Peptide counts for all parsed files on erda (for data selection)
fn_rawfile_metadata: str = 'data/files_selected_metadata.csv' # Machine parsed metadata from rawfile workflow
M: int = 1000 # M most common features
MIN_SAMPLE: typing.Union[int, float] = 0.5 # Minimum number or fraction of total requested features per Sample
index_col: typing.Union[str,int] = 'Sample ID' # Can be either a string or position (typical 0 for first column)
# query expression for subsetting
query_subset_meta: str = "`instrument serial number` in ['Exactive Series slot #6070',]" # query for metadata, see files_selected_per_instrument_counts.csv for options
logarithm: str = 'log2' # Log transformation of initial data (select one of the existing in numpy)
experiment_folder: str = 'runs/experiment_03'
columns_name: str = 'peptide'

In [ ]:
# There must be a better way...
@dataclass
class DataConfig:
    """Documentation. Copy pasted arguments to a dataclass."""
    FN_PEPTIDE_INTENSITIES: str = 'data/df_intensities_N07813_M10000.csv'  # Samples metadata extraced from erda
    FN_PEPTIDE_FREQ: str = 'data/processed/count_all_peptides.json' # Peptide counts for all parsed files on erda (for data selection)
    fn_rawfile_metadata: str = 'data/rawfile_metadata.csv' # Machine parsed metadata from rawfile workflow
    M: int = 1000 # M most common features
    MIN_SAMPLE: typing.Union[int, float] = 0.5 # Minimum number or fraction of total requested features per Sample
    index_col: typing.Union[
        str, int
    ] = "Sample ID"  # Can be either a string or position (typical 0 for first column)
    # query expression for subsetting
    query_subset_meta: str = "`instrument serial number` in ['Exactive Series slot #6070',]" # query for metadata, see files_selected_per_instrument_counts.csv for options
    logarithm: str = 'log2' # Log transformation of initial data (select one of the existing in numpy)
    experiment_folder: str = 'runs/experiment_03'
    columns_name: str = "peptide"


params = DataConfig(
    FN_PEPTIDE_INTENSITIES=FN_PEPTIDE_INTENSITIES,
    FN_PEPTIDE_FREQ=FN_PEPTIDE_FREQ,
    fn_rawfile_metadata=fn_rawfile_metadata,
    M=M,
    index_col=index_col,
    query_subset_meta=query_subset_meta,
    logarithm=logarithm,
    experiment_folder=experiment_folder,
    columns_name=columns_name
)

params = OmegaConf.create(params.__dict__)
dict(params)

process arguments

In [ ]:
logger.info(f"{FN_PEPTIDE_INTENSITIES = }")

## Setup

In [ ]:
# if not experiment_folder:
#     experiment_folder = query_subset_meta.replace('_', ' ')
#     experiment_folder = parse_query_expression(experiment_folder)
#     experiment_folder = experiment_folder.strip()
#     experiment_folder = experiment_folder.replace(' ', '_')
#     params.experiment_folder
experiment_folder = Path(experiment_folder)
experiment_folder.mkdir(exist_ok=True, parents=True)
logger.info(f'Folder for output = {experiment_folder}')

### Select M most common features

In [ ]:
import json
from collections import Counter
# Use PeptideCounter instead?
with open(Path(params.FN_PEPTIDE_FREQ)) as f:
    freq_pep_all = Counter(json.load(f)['counter'])
    
selected_peptides = {k: v for k, v in freq_pep_all.most_common(params.M)}
print(f"No. of selected features: {len(selected_peptides):,d}")

## Raw data

In [ ]:
%%time
params.used_features = sorted(selected_peptides)
if isinstance(params.index_col, str): params.used_features.insert(0, params.index_col)
analysis = AnalyzePeptides.from_csv(fname=params.FN_PEPTIDE_INTENSITIES,
                                     nrows=None,
                                     index_col=params.index_col,
                                     usecols=params.used_features
                                    )
analysis.df.columns.name = params.columns_name

log_fct = getattr(np, params.logarithm)
analysis.log_transform(log_fct)
logger.info(f"{analysis = }")

### Sample file name cleaning and Sample selection

Rename some samples
- [ ] needs to be moved into the data extraction pipeline from the server

In [ ]:
# # some date are not possible in the indices
# rename_indices_w_wrong_dates = {'20161131_LUMOS1_nLC13_AH_MNT_HeLa_long_03': '20161130_LUMOS1_nLC13_AH_MNT_HeLa_long_03',
#                                 '20180230_QE10_nLC0_MR_QC_MNT_Hela_12': '20180330_QE10_nLC0_MR_QC_MNT_Hela_12',
#                                 '20161131_LUMOS1_nLC13_AH_MNT_HeLa_long_01': '20161130_LUMOS1_nLC13_AH_MNT_HeLa_long_01',
#                                 '20180230_QE10_nLC0_MR_QC_MNT_Hela_11': '20180330_QE10_nLC0_MR_QC_MNT_Hela_11',
#                                 '20161131_LUMOS1_nLC13_AH_MNT_HeLa_long_02': '20161130_LUMOS1_nLC13_AH_MNT_HeLa_long_02'}
# analysis.df.rename(index=rename_indices_w_wrong_dates, inplace=True)

In [ ]:
assert analysis.df.index.is_unique, "Duplicates in index"
analysis.df.sort_index(inplace=True)

Select samples based on completeness

In [ ]:
if isinstance(params.MIN_SAMPLE, float):
    msg = f'Fraction of minimum sample completeness over all features specified with: {params.MIN_SAMPLE}\n'
    # assumes df in wide format
    params.MIN_SAMPLE = int(analysis.df.shape[1] * params.MIN_SAMPLE)
    msg += f'This translates to a minimum number of total samples: {params.MIN_SAMPLE}'
    print(msg)

mask = analysis.df.notna().sum(axis=1) > params.MIN_SAMPLE
msg = f'Drop {len(mask) - mask.sum()} of {len(mask)} initial samples.'
print(msg)
analysis.df = analysis.df.loc[mask]

In [ ]:
params.used_samples = analysis.df.index.to_list()

## Machine metadata

- read from file using [ThermoRawFileParser](https://github.com/compomics/ThermoRawFileParser)

In [ ]:
df_meta = pd.read_csv('data/files_selected_metadata.csv', index_col=0)
date_col = 'Content Creation Date'
df_meta[date_col] = pd.to_datetime(df_meta[date_col])
df_meta.head()

In [ ]:
cols_instrument = ['Thermo Scientific instrument model',
                          'instrument attribute',
                          'instrument serial number', ]
df_meta.groupby(cols_instrument)[date_col].agg(['count','min','max']) 

In [ ]:
meta_raw_selected = [
 'Content Creation Date', 
 'Thermo Scientific instrument model',
 'instrument serial number',
 'Software Version', 
 'Number of MS1 spectra',
 'Number of MS2 spectra', 
 'Number of scans',
 'MS max charge',
 'MS max RT',
 'MS min MZ',
 'MS max MZ',
 'MS scan range', 
 'mass resolution',
 'Retention time range',
 'Mz range',
 'beam-type collision-induced dissociation', 
 'injection volume setting',
 'dilution factor',
]
df_meta[meta_raw_selected].describe(datetime_is_numeric=True)

In [ ]:
df_meta = df_meta.query(query_subset_meta) # can be based on table above
df_meta[meta_raw_selected].describe(datetime_is_numeric=True, percentiles=np.linspace(0.05, 0.95, 10))

set a minimum retention time

In [ ]:
min_RT_max = 120 # minutes
msg = f"Minimum RT time maxiumum is set to {min_RT_max} minutes (to exclude too short runs, which are potentially fractions)."
mask_RT = df_meta['MS max RT'] >= 120 # can be integrated into query string
msg += f" Total number of samples retained: {int(mask_RT.sum())}."
print(msg)

In [ ]:
df_meta = df_meta.loc[mask_RT]
df_meta.sort_values(date_col, inplace=True)

In [ ]:
meta_stats = df_meta.describe(include='all', datetime_is_numeric=True)
meta_stats

subset with variation

In [ ]:
meta_stats.loc[:, (meta_stats.loc['unique'] > 1) |  (meta_stats.loc['std'] > 0.1)]

check some columns describing settings
  - quite some variation due to `MS max charge`: Is it a parameter?

In [ ]:
meta_raw_settings = [
 'Thermo Scientific instrument model',
 'instrument serial number',
 'Software Version', 
 'MS max charge',
 'mass resolution',
 'beam-type collision-induced dissociation', 
 'injection volume setting',
 'dilution factor',
]
df_meta[meta_raw_settings].drop_duplicates() # index gives first example with this combination

view without `MS max charge`:
  - software can be updated
  - variation by `injection volume setting` and instrument over time
  - missing `dilution factor`
  

In [ ]:
to_drop = ['MS max charge']
df_meta[meta_raw_settings].drop(to_drop, axis=1).drop_duplicates() # index gives first example with this combination

- check for variation in `software Version` and `injection volume setting`

In [ ]:
try:
    analysis.df = analysis.df.loc[df_meta.index]
except KeyError as e:
    logger.warning(e)
    logger.warning("Ignore missing samples in quantified samples")
    analysis.df = analysis.df.loc[analysis.df.index.intersection(df_meta.index)]

analysis.df_meta = df_meta

### Interactive and Single plots

Scatter plots need to become interactive.

In [ ]:
K = 2
pcs = analysis.get_PCA(n_components=K) # should be renamed to get_PCs
pcs = pcs.iloc[:,:K].join(analysis.df_meta)

pcs_name = pcs.columns[:2]
pcs = pcs.reset_index()
pcs.head()

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))
analyzers.seaborn_scatter(pcs[pcs_name], fig, ax, meta=pcs['Thermo Scientific instrument model'])

In [ ]:
fig = px.scatter(
    pcs, x=pcs_name[0], y=pcs_name[1],
    hover_name='Sample ID',
    hover_data=meta_raw_selected,
    title=f'First two Principal Components of {analysis.M} most abundant peptides for {pcs.shape[0]} samples',
    color="Thermo Scientific instrument model",
    width=1200,
    height=600
)
fig

## Sample Medians and percentiles

- see boxplot [function in R](https://github.dev/symbioticMe/proBatch/blob/8fae15049be67693bd0d4a4383b51bfb4fb287a6/R/initial_assessment.R#L199-L317), which is used for [publication figure](https://github.dev/symbioticMe/batch_effects_workflow_code/blob/696eb609a55ba9ece68b732e616d7ebeaa660373/AgingMice_study/analysis_AgingMice/5b_Fig2_initial_assessment_normalization.R)
- check boxplot functions: [bokeh](https://docs.bokeh.org/en/latest/docs/gallery/boxplot.html), [plotly](https://plotly.com/python/box-plots/), [eventplot](https://matplotlib.org/stable/gallery/lines_bars_and_markers/eventplot_demo.html#sphx-glr-gallery-lines-bars-and-markers-eventplot-demo-py)

In [ ]:
# analysis.df.iloc[:100].T.boxplot(rot=90, backend=None, figsize=None)
# analysis.df.boxplot()
analysis.df.head()

In [ ]:
df = analysis.df
df = df.join(df_meta[date_col])
df = df.set_index(date_col).sort_index().to_period('min').T
df

In [ ]:
ax = df.boxplot(rot=80, figsize=(20, 10), fontsize='large', showfliers=False, showcaps=False)
_ = vaep.plotting.select_xticks(ax)

Plot sample median over time
  - check if points are equally spaced (probably QC samples are run in close proximity)
  - the machine will be not use for intermediate periods

In [ ]:
dates = df_meta[date_col].sort_values()
dates.name = 'date'
median_sample_intensity = (analysis.df
                           .median(axis=1)
                           .to_frame('median intensity'))
median_sample_intensity = median_sample_intensity.join(dates)

ax = median_sample_intensity.plot.scatter(x='date', y='median intensity', rot=90,  fontsize='large', figsize=(20, 10),
                                          xticks=vaep.plotting.select_dates(
                                              median_sample_intensity['date'])
                                          )

- the closer the labels are there denser the samples are measured aroudn that time.

## Split: Train, validation and to_periodata

- test data is in clinical language often denoted as independent validation cohort
- validation data (for model)

In [ ]:
analysis.splits = DataSplits(is_wide_format=True)
splits = analysis.splits
print(f"{analysis.splits = }")
analysis.splits.__annotations__

In [ ]:
analysis.df

In [ ]:
percentiles = (0.8, 0.9)  # change here

percent_str = [f'{int(x*100)}%' for x in percentiles]
split_at_date = analysis.df_meta[date_col].describe(
    datetime_is_numeric=True, percentiles=(0.8, 0.9)).loc[percent_str]
split_at_date = tuple(pd.Timestamp(t.date()) for t in split_at_date)

print(f"{split_at_date[0] = }", f"{split_at_date[1] = }", sep="\n")

In [ ]:
idx_train = analysis.df_meta[date_col] < split_at_date[0]
analysis.splits.train_X = analysis.df.loc[idx_train]
analysis.splits.train_X

In [ ]:
idx_validation = ((analysis.df_meta[date_col] >= split_at_date[0]) & (
    analysis.df_meta[date_col] < split_at_date[1]))
analysis.splits.val_X = analysis.df.loc[idx_validation]
analysis.splits.val_X

In [ ]:
idx_test = (analysis.df_meta[date_col] >= split_at_date[1])
# analysis.df_test =
analysis.splits.test_X = analysis.df.loc[idx_test]
analysis.splits.test_X

In [ ]:
idx_test_na = analysis.splits.test_X.stack(
    dropna=False).loc[splits.test_X.isna().stack()].index
print(f"number of missing values in test data: {len(idx_test_na)}")

## Peptide frequency  in training data

- higher count, higher probability to be sampled into training data
- missing peptides are sampled both into training as well as into validation dataset
- everything not in training data is validation data

Based on unmodified training data

In [ ]:
# analysis.splits.to_wide_format()
assert analysis.splits is splits, "Sanity check failed."

In [ ]:
freq_per_peptide = feature_frequency(analysis.splits.train_X)
freq_per_peptide

In [ ]:
msg = "Total number of samples in training data split: {}"
print(msg.format(len(analysis.splits.train_X)))

In [ ]:
folder = experiment_folder / 'data' 
folder.mkdir(parents=True, exist_ok=True)
freq_per_peptide.to_csv(folder / 'freq_train.csv')

Conserning sampling with frequency weights:
  - larger weight -> higher probablility of being sampled
  - weights need to be alignable to index of original DataFrame before grouping (same index)

## Sample targets (Fake NAs)

Add goldstandard targets for valiation and test data
- based on same day
- same instrument

Create some target values by sampling 5% of the validation and test data.

In [ ]:
analysis.splits.to_long_format(name_values='intensity') # long format as sample_data uses long-format

In [ ]:
splits.val_X, splits.val_y = sample_data(splits.val_X, sample_index_to_drop=0, weights=freq_per_peptide)
splits.test_X, splits.test_y = sample_data(splits.test_X, sample_index_to_drop=0, weights=freq_per_peptide)

for k, s in splits:
    s.sort_index(inplace=True)

## Save in long format

- Data in long format: (peptide, sample_id, intensity)
- no missing values kept

In [ ]:
splits.dump(folder=folder)  # dumps data in long-format

In [ ]:
# # Reload from disk
# splits = DataSplits.from_folder(folder)

## PCA plot of training data

In [ ]:
ana_train_X = analyzers.AnalyzePeptides(data=splits.train_X, is_wide_format=False, ind_unstack='peptide')
figures['pca_train'] = ana_train_X.plot_pca()
vaep.savefig(figures['pca_train'], experiment_folder / f'pca_plot_raw_data_{ana_train_X.fname_stub}')

In [ ]:
# add to DataSplits a inputs attribute

data_dict = {'train': splits.train_X, 'valid': splits.val_X, 'test': splits.test_X}
PCs = pd.DataFrame()
split_map = pd.Series(dtype='string')
for key, df in data_dict.items():
    df = df.unstack()
    PCs = PCs.append(ana_train_X.calculate_PCs(df))
    split_map = split_map.append(pd.Series(key, index=df.index))

fig, ax = plt.subplots(figsize=(15,8))
ax.legend(title='splits')
analyzers.seaborn_scatter(PCs.iloc[:, :2], fig, ax, meta=split_map,
                          title='First two principal compements (based on training data PCA)')
ax.get_legend().set_title("split")

For *Collaborative Filtering*, new samples could be initialized based on a KNN approach in the original sample space or the reduced PCA dimension.
  - The sample embeddings of the K neighearst neighbours could be averaged for a new sample

In [ ]:
# Optional: Change number of principal components
# K = 2
# _ = ana_train_X.get_PCA(n_components=K)

train_PCs = ana_train_X.calculate_PCs(splits.train_X.unstack())
test_PCs = ana_train_X.calculate_PCs(splits.test_X.unstack())
nn = NearestNeighbors(n_neighbors=5).fit(train_PCs)

Select K neareast neighbors for first test data sample from training data. Compare equal distance mean to mean weighted by distances.

In [ ]:
d, idx = nn.kneighbors(test_PCs.iloc[1:2])
# test_PCs.iloc[1]
idx

In [ ]:
train_PCs.iloc[idx[0]]

In [ ]:
w = d / d.sum()
display(f"Sample weights based on distances: {w = }")
w.flatten().reshape(5,1) * train_PCs.iloc[idx[0]] # apply weights to values

In [ ]:
pd.DataFrame( (train_PCs.iloc[idx[0]].mean(), # mean
              (w.flatten().reshape(5,1) * train_PCs.iloc[idx[0]]).sum() # sum of weighted samples
              ), index=['mean','weighted by distance '])

Add visual representation of picked points in the first two principal components

In [ ]:
ax.scatter(x=test_PCs.iloc[1]['PC 1'], y=test_PCs.iloc[1]['PC 2'], s=100, marker="v", c='r')
ax.scatter(x=train_PCs.iloc[idx[0]]['PC 1'], y=train_PCs.iloc[idx[0]]['PC 2'], s=100, marker="s", c='y')
fig

## Digression on MultiIndex: Data Selection

- use mulitindex for obtaining validation split

[[stackoverflow](https://stackoverflow.com/questions/53927460/select-rows-in-pandas-multiindex-dataframe), [guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)]

- [`xs` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.xs.html) or [`pd.IndexSlice`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.IndexSlice.html?highlight=indexslice)

In [ ]:
sample_name_1, sample_name_2 = analysis.df_long.sample(2).index.get_level_values(-1).to_list()
sample_name_1, sample_name_2

In [ ]:
analysis.df_long.head()

In [ ]:
analysis.df_long.loc[pd.IndexSlice[:, sample_name_1], :]

In [ ]:
analysis.df_long.loc[(slice(None), sample_name_2), :]

with a series the syntax changes slightly (no column) and the indexing behaviour different if a string or a list is passed:

In [ ]:
s = analysis.df_long.squeeze()
s

In [ ]:
s.loc[pd.IndexSlice[:, sample_name_2]]

In [ ]:
s.loc[pd.IndexSlice[:, [sample_name_2]]]

## Save parameters

In [ ]:
print(OmegaConf.to_yaml(params))

In [ ]:
with open(experiment_folder/'data_config.yaml', 'w') as f:
    OmegaConf.save(params, f)